In [26]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
df = pd.read_csv('recipe_list.csv')
df.head()

,레시피id,사진,요리,재료,조리법,카테고리
0,1,NaN,스팸 김밥,스팸 1통\n밥 2그릇\n달걀 4개\n맛소금\n김밥용 김 2장,"1. 밥에 참기름과 맛소금을 넣고 잘 버무려 주세요. 밥 두공기에, 맛소금 1/2 ...","일식, 밥, 김밥, 분식, 돼지고기"
1,2,NaN,떡볶이,쌀떡 400g \n사각어묵 3장 \n비엔나 소세지 3개 \n삶은 달걀 2개 \n대파...,1. 떡은 미리 찬물에 2-3번 헹군 다음에 이렇게 담가두세요. 다른 재료를 준비하...,"분식, 떡, 매콤"
2,3,NaN,핫도그,모닝빵 \n프랑크소세지 2개\n슬라이스 치즈 1장\n양파 반개\n양배추 채 적당히 ...,1. 우선 양파는 얇게 썰어줍니다\n2. 채썬 양파는 물에 담궈서 매운맛을 빼줍니다...,"양식, 간단, 샌드위치, 돼지고기"
3,4,NaN,김치찌개,돼지고기 삼겹살 300g\n신김치 송송썰어 3컵 분량 (약 300g)\n쌀뜨물 3컵...,1. 오늘 육수는 쌀뜨물을 약 3~4컵 분량 준비해 둡니다. 쌀뜨물 대신 멸치육수를...,"한식, 매콤, 고기, 국, 김치, 돼지고기"
4,5,NaN,김치말이국수,소면 100g \n잘익은 김치 80g \n삶은 계란 1개 \n오이 1/6개\n김치국...,1. 김치는 신걸로 준비해야 맛있습니다 양념을 털어내고 잘게 썰어주세요 설탕 0.3...,"한식, 매콤, 김치, 면, 분식"


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
df['재료'] = df['재료'].fillna('')
tfidf_matrix = tfidf.fit_transform(df['재료'])

In [29]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [30]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['요리']).drop_duplicates()

In [31]:
def get_recommendations(title, cosine_sim=cosine_sim):
    
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:3]
    food_indices = [i[0] for i in sim_scores]
    return df['요리'].iloc[food_indices]

In [32]:
df_new1 = df[['요리', '카테고리']]

In [33]:
# 데이터 내의 빈칸 처리
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [34]:
# Apply clean_data function to your features.
features = ['요리', '카테고리']

for feature in features:
    df_new1[feature] = df_new1[feature].apply(clean_data)   

C:\Users\User\AppData\Local\Temp/ipykernel_36332/887757627.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new1[feature] = df_new1[feature].apply(clean_data)


In [35]:
def create_soup(x):
    return  x['요리'] + ' ' + x['카테고리']
df100 = df_new1.copy()        
df_new1['정제'] = df_new1.apply(create_soup, axis=1)
df_new1['정제']

C:\Users\User\AppData\Local\Temp/ipykernel_36332/3560293087.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new1['정제'] = df_new1.apply(create_soup, axis=1)


0        스팸김밥 일식,밥,김밥,분식,돼지고기
1                 떡볶이 분식,떡,매콤
2         핫도그 양식,간단,샌드위치,돼지고기
3     김치찌개 한식,매콤,고기,국,김치,돼지고기
4        김치말이국수 한식,매콤,김치,면,분식
5           돈까스 일식,고기,튀김,돼지고기
6        김치잔치국수 한식,매콤,김치,면,분식
7          햄버거 양식,고기,샌드위치,소고기
8           소불고기 한식,고기,달콤,소고기
9                대패육전 한식,전,고기
10         투움바파스타 양식,면,파스타,해물
11          비소캐 한식,고기,달콤,돼지고기
12             고구마맛탕 한식,달콤,튀김
13          프렌치토스트 양식,간단,샌드위치
14           김치볶음밥 한식,김치,밥,매콤
15               옥수수콘치즈 양식,치즈
16       순두부열라면 한식,라면,분식,면,매콤
17            감바스 양식,새우,빵,해산물
18              계란볶음밥 중식,밥,계란
19        크림스파게티 양식,파스타,면,해산물
20          육회 한식,고기,달콤,회,소고기
21        돼지고기카레 일식,밥,고기,돼지고기
22           팟타이 아시아,면,해산물,이색
23                오코노미야키 일식,전
24               연어초밥 일식,초밥,회
25                 물회 회,한식,매콤
26       탕수육 중식,튀김,달콤,고기,돼지고기
27               멘보샤 중식,튀김,해물
28               현미도넛 빵,양식,간단
29                감자전 한식,전,이색
30             피시엔칩스 양식,튀김,해물
31              쫄면 한식,분식,면,매콤
32                일본식냉라면 일식,면
33        

In [36]:
df_new2 = df_new1
# CountVectorizer를 통해서 진행, 불용어 제거
from sklearn.feature_extraction.text import CountVectorizer
stopwords=['으로', '가', '을', '를', '에', '로', '에게', '또한', ',', '.']

count = CountVectorizer(stop_words=stopwords)
count_matrix = count.fit_transform(df_new2['정제'])
count_matrix

<52x81 sparse matrix of type '<class 'numpy.int64'>'
	with 212 stored elements in Compressed Sparse Row format>

In [37]:
from sklearn.metrics.pairwise import cosine_similarity
# 다른 cosine_similarity를 구해준다, 이것은 category 에 대한 matrix
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
cosine_sim2

array([[1.        , 0.25819889, 0.2       , ..., 0.2       , 0.18257419,
        0.2       ],
       [0.25819889, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.2       , 0.        , 1.        , ..., 0.        , 0.36514837,
        0.4       ],
       ...,
       [0.2       , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.18257419, 0.        , 0.36514837, ..., 0.        , 1.        ,
        0.73029674],
       [0.2       , 0.        , 0.4       , ..., 0.        , 0.73029674,
        1.        ]])

In [38]:
df_new1 = df_new1.reset_index()
indices = pd.Series(df_new1.index, index=df_new1['요리'])

In [39]:
get_recommendations('돈까스', cosine_sim2)

21    돼지고기 카레
26        탕수육
Name: 요리, dtype: object